In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
((train_input, train_target),
     (test_input, test_target)) =tf.keras.datasets.fashion_mnist.load_data()

In [3]:
train_input_scale = (train_input / 255)
test_input_scale = (test_input / 255)

In [4]:
inputs = tf.keras.layers.Input(shape=(28, 28,1))
x = tf.keras.layers.Resizing(224,224)(inputs)
x = tf.keras.layers.Conv2D(
    64, kernel_size = 3,
    padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(
    128, kernel_size = 3,
    padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(
    256, kernel_size = 3,
    padding="same", strides=1)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(
    512, kernel_size = 3,
    padding="same", strides=1)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dense(
    10, activation="softmax")(x)
model = tf.keras.Model(inputs, x)
model.summary()

model.compile(
    loss = "sparse_categorical_crossentropy", metrics=["acc"])
hist = model.fit(
    train_input_scale, train_target,
    epochs=10, batch_size=32)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,683,722 (6.42 MB)

 Trainable params: 1,683,722 (6.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 242s 121ms/step - acc: 0.3528 - loss: 1.7842
Epoch 2/10
1817/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 122ms/step - acc: 0.6126 - loss: 1.0541

KeyboardInterrupt: 

전이학습은 이미 학습된 모델을 바탕으로 활용하여 내 모델을 만드는 과정

In [4]:
tf.keras.applications.EfficientNetB0(include_top=True).summary()

In [1]:
import cv2, tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
((train_input, train_target),
     (test_input, test_target)) =tf.keras.datasets.fashion_mnist.load_data()

tr_x = np.array([cv2.resize(i,(224,224)) for i in tqdm.tqdm(train_input[::30])]) #
tr_x = np.array([cv2.cvtColor(i, cv2.COLOR_GRAY2BGR) for i in tr_x]) # gray to rgb
#tr_x = (tr_x / 255)
backbone = tf.keras.applications.EfficientNetB0(include_top=False)
backbone.trainable = False

100%|██████████| 2000/2000 [00:00<00:00, 18383.37it/s]


In [3]:
inputs = tf.keras.layers.Input((224,224, 3))
x = backbone(inputs)
x = tf.keras.layers.GlobalAvgPool2D()(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

In [4]:
model.compile(
    loss = "sparse_categorical_crossentropy", metrics=["acc"])
hist = model.fit(
    tr_x, train_target[::30],
    epochs=10, batch_size=32)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 45s 290ms/step - acc: 0.5671 - loss: 1.3820
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - acc: 0.8126 - loss: 0.5632
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - acc: 0.8511 - loss: 0.4364
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - acc: 0.8728 - loss: 0.3827
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - acc: 0.8706 - loss: 0.3525
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - acc: 0.8915 - loss: 0.3305
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - acc: 0.8808 - loss: 0.3326
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - acc: 0.8902 - loss: 0.3177
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - acc: 0.9061 - loss: 0.2837
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - acc: 0.9061 - loss: 0.2768
